# Sparse Hebbian Learning: toward a quantitative measure of the quality of filters

We are interested here in learning the "optimal" components of a set of images (let's say some "natural", usual images). As there is no supervisor to guide the learning, this is called unsupervised learning. Our basic hypothesis to find the best ("optimal") components will be to assume that *a priori* the most sparse is more plausible. We will implement the derived algorithm in this set of scripts.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import pandas as pd

## experiments

To test and control for the role of different parameters, we will have a first object (in the [shl_experiments.py](https://github.com/bicv/SHL_scripts/blob/master/shl_scripts/shl_experiments.py) script) that controls a learning experiment. It contains all relevant parameters, but can also keep a trace of the history of some statistics. This is useful to compare the relative efficiency of the different solutions.


In [3]:
do_random = True # draw new coeff at random
do_random = False # draw new coeff with bootstrap resampling?

do_high = False # do a surrogate with all coeffs
do_high = True # do a surrogate with the higher coeffs

do_double_shuffle = False # only shuffles with each sample 
do_double_shuffle = True # shuffle accross dictionary elements

In [4]:
tag = 'coding'
homeo_methods = ['None', 'HAP', 'HEH']

record_num_batches = 2**12

seed = 42
np.random.seed(seed)

from shl_scripts.shl_experiments import SHL
shl = SHL()
data = shl.get_data(matname=tag + '_test')
indx = np.random.permutation(data.shape[0])[:record_num_batches]

list_figures = []

dico = {}
for homeo_method in homeo_methods:
    print(15*'🐶' + homeo_method[:3] + 15*'🐶')
    shl = SHL(homeo_method=homeo_method)
    dico[homeo_method] = shl.learn_dico(data=data, list_figures=list_figures, matname=tag + '_' + homeo_method)


the data extraction is locked data_cache/coding_test_data


AttributeError: 'str' object has no attribute 'shape'

## coding

The learning itself is done via a gradient descent but is highly dependent on the coding / decoding algorithm. This belongs to a another function (in the [shl_encode.py](https://github.com/bicv/SHL_scripts/blob/master/shl_scripts/shl_encode.py) script)

In [ ]:
from shl_scripts.shl_encode import sparse_encode
stick = np.arange(shl.n_dictionary)*shl.nb_quant
P_cum_zeroeffect = np.linspace(0, 1, shl.nb_quant, endpoint=True)[np.newaxis, :] * np.ones((shl.n_dictionary, 1))

for homeo_method in homeo_methods:
    print(15*'🐶' + homeo_method[:3] + 15*'🐶')

    shl = SHL(homeo_method=homeo_method)
    sparse_code = sparse_encode(data[indx, :], dico[homeo_method].dictionary, P_cum=dico[homeo_method].P_cum, C=shl.C, 
                                     l0_sparseness=shl.l0_sparseness, gain=None)   

    from shl_scripts.shl_tools import print_stats
    SD, SE = print_stats(data[indx, :], dico[homeo_method].dictionary, sparse_code)

## Generating new coefficients by shuffling and decoding

In [ ]:
N_show = 30

def shuffling(data, sparse_code, dico, N_show=N_show):
    if do_random:
        from shl_scripts.shl_encode import inv_quantile, inv_rescaling
        sparse_code_bar = inv_rescaling(inv_quantile(dico.P_cum, np.random.rand(sparse_code.shape[0], sparse_code.shape[1])), C=shl.C)
    else:
        sparse_code = sparse_encode(data, dico.dictionary, P_cum=dico.P_cum, C=shl.C, 
                                     l0_sparseness=dico.n_dictionary, gain=None)   

        sparse_code_bar = sparse_code.copy()
        
        #sparse_code_bar = sparse_code_bar.T
        #np.random.shuffle(sparse_code_bar)
        #sparse_code_bar = sparse_code_bar.T
        sparse_code_bar = np.random.permutation(sparse_code_bar)
        
    if do_double_shuffle:
        sparse_code_bar = np.random.permutation(sparse_code_bar.ravel()).reshape(sparse_code_bar.shape)
        #np.random.shuffle(sparse_code_bar)
    print('sparse_code_bar')
    plt.matshow(sparse_code_bar[:N_show, :])
    plt.show()

    def threshold_mask(sparse_code, l0_sparseness, n_dictionary):
        thr = np.percentile(sparse_code, 100 * (1 - l0_sparseness/n_dictionary), axis=1)
        return (sparse_code>thr[:, np.newaxis])

    sparse_code_bar_high = threshold_mask(sparse_code_bar, shl.l0_sparseness, shl.n_dictionary) * sparse_code_bar
    print('sparse_code_bar_high')
    plt.matshow(sparse_code_bar_high[:N_show, :])
    plt.show()
    return sparse_code_bar, sparse_code_bar_high

def pipeline(sparse_code_bar, sparse_code_bar_high, dico, index, N_show=N_show):

    if do_high:
        patches_bar = sparse_code_bar_high @ dico.dictionary
    else:
        patches_bar = sparse_code_bar @ dico.dictionary

        
    SD = np.sqrt(np.mean(patches_bar**2, axis=1))

    sparse_code_rec = sparse_encode(patches_bar, dico.dictionary, P_cum=dico.P_cum, C=shl.C, 
                                     l0_sparseness=shl.l0_sparseness, gain=None)   

    print('bar: average non-zeros', np.count_nonzero(sparse_code_bar, axis=0)[:N_show])
    print('bar_high: average non-zeros', np.count_nonzero(sparse_code_bar_high, axis=0)[:N_show])
    print('rec: average non-zeros', np.count_nonzero(sparse_code_rec, axis=0)[:N_show])
    
    from shl_scripts import print_stats
    SD, SE = print_stats(patches_bar, dico.dictionary, sparse_code_rec, verbose=False, display=True, N_show=N_show)
    #plt.matshow(sparse_code_rec[:N_show, :])
    plt.show()

    print('mean deviation of coefficients = ', np.mean(np.abs(sparse_code_bar)), np.mean(np.abs(sparse_code_bar_high)), np.mean(np.abs(sparse_code_rec)))
    print('mean deviation of coefficient errors = ', np.mean(np.abs(sparse_code_bar_high-sparse_code_rec)))

    from shl_scripts.shl_encode import quantile, rescaling

    q_rec = quantile(dico.P_cum, rescaling(sparse_code_rec, C=shl.C), stick, do_fast=False)
    q_bar = quantile(dico.P_cum, rescaling(sparse_code_bar_high, C=shl.C), stick, do_fast=False)

    print('mean deviation of quantiles = ', np.mean(np.abs(q_bar)))
    print('mean deviation of quantiles = ', np.mean(np.abs(q_rec)))
    print('total deviation of quantiles = ', np.mean(np.abs(q_bar-q_rec)))
    print('ratio deviation of quantiles = ', np.mean(np.abs(q_bar-q_rec))/np.mean(np.abs(q_bar)))
    aerror = np.mean(np.abs(q_bar-q_rec))/np.mean(np.abs(q_bar))

    perror = 1 - np.mean( (sparse_code_bar>0) == (sparse_code_rec>0))
    print('proba incorrect coefficients = ', perror)

    perror_high = 1 - np.mean( (sparse_code_bar_high > 0) == (sparse_code_rec>0))
    print('proba incorrect coefficients (high) = ', perror_high)
    
    return pd.DataFrame({'error':[(SD/SE).mean()],
                               'aerror':[aerror],
                               'perror':[perror],
                               'perror_high':[perror_high]
                                        },
                                index=[index])

record = None
for homeo_method in homeo_methods:
    print(42*'🐶')
    print(19*'🐶' + '  ' + homeo_method + '  ' + 19*'🐶')
    print(42*'🐶')

    shl = SHL(homeo_method=homeo_method)

    sparse_code_bar, sparse_code_bar_high = shuffling(data[indx, :], sparse_code, dico[homeo_method])
    record_ = pipeline(sparse_code_bar, sparse_code_bar_high, dico[homeo_method], index=homeo_method)
    if record is None:
        record = record_
    else:
        record = pd.concat((record, record_))


In [ ]:
record

In [ ]:
from shl_scripts.shl_tools import get_perror
for homeo_method in homeo_methods:
    shl = SHL(homeo_method=homeo_method)
    print ('homeo_method', homeo_method, ', perrror=', get_perror(data[indx, :], dico[homeo_method].dictionary,  None,
                            algorithm='mp', fit_tol=None,
                             P_cum=dico[homeo_method].P_cum, gain=None,
                             C=shl.C, do_sym=shl.do_sym,
                             l0_sparseness=shl.l0_sparseness))


## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts, pandas